# seeded LDA 결과를 기반으로 multi labeling 수행하는 코드

In [ ]:
import pandas as pd

In [ ]:
everytime_keywords = pd.read_excel('/content/drive/MyDrive/2023_연구학점/jiae/everytime_keywords2.xlsx') # every time 데이터셋 원본문서 + 형태소 추출 문서
seededlda = pd.read_excel('/content/drive/MyDrive/2023_연구학점/jiae/seeded LDA/0603_everytime2.xlsx') # seeded LDA 결과
dep_word = pd.read_csv('/content/drive/MyDrive/2023_연구학점/jiae/dep_word_cleaned.csv') # 우울 단어 사전

In [ ]:
everytime_keywords

,Unnamed: 0,text,keyword
0,0,자꾸 아 이건 다들 초등학교때 배웠을 텐데~ 하면서 설명 생략하시는데\n그거 고딩 ...,"자꾸,이건,다,들다,초등학교,때,배우다,하다,설명,생략,하다,그거,고딩,화,과정,늘..."
1,1,시험이 11월에 끝나는 ㅂㅅ이 있다?,"시험,끝나다,이,있다"
2,2,연애하고 싶다 이런건 아니고 (이미 남친 있음)\n\n최근에 축하 받을 일 생겼는데...,"연애,싶다,이렇다,아니다,이미,남친,있다,최근,축하,받다,일,생기다,남친,포항,있다..."
3,3,이제 복전이라고 뜨게 주잖아... 그래도 회사에서 볼 때는 복전이라도 상경학위 있는...,"이제,복전,뜨다,주다,그래도,회사,볼,때,복전,상경,학위,있다,더,낫다,물론,스펙,..."
4,4,오뚜기 정신을 본받아 숱한 실패에도 마음을 다잡고 다시 도전하는 사람이 멋진 사람이...,"오뚜기,정신,본받다,숱,실패,마음,다잡다,다시,도전,하다,사람,멋지다,사람,멋지다,..."
...,...,...,...
1193,1193,혹시 이런 기분 드는 사람 있어? 혼자 떨어져서 샤워할 때나 목욕할 때 드는데 갑자...,"혹시,이렇다,기분,들다,사람,있다,혼자,떨어지다,샤워,하다,때,목욕,하다,때,들다,..."
1194,1194,혹시 중도휴학도 신청할 수 있는 시기가 따로 있어?? 지금 2학기에 가게될 군대도 ...,"혹시,중도,휴학,신청,하다,수,있다,시기,따로,있다,지금,학기,가게,되다,군대,빼다..."
1195,1195,확인 강박증이 점점 심해져 현관문을 닫았는지 확인만 몇번해 내생각에 옛날에 집에 누...,"확인,강박증,점점,심해지다,현관문,닫다,확인,몇번,하다,생각,옛날,집,누가,들어오다..."
1196,1196,휴학하고 에타 처음 들어온다하는커뮤도 딱히 바쁜친구들에게 시덥잖은 얘기하기도그래서 ...,"휴학,에타,처음,들어오다,커뮤,딱하다,바쁘다,친구,시,덥다,얘기,하다,그래서,여기,..."


In [ ]:
everytime_keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1198 entries, 0 to 1197
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1198 non-null   int64 
 1   text        1198 non-null   object
 2   keyword     1163 non-null   object
dtypes: int64(1), object(2)
memory usage: 28.2+ KB


In [ ]:
everytime_keywords = everytime_keywords.fillna(' ')

In [ ]:
everytime_keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1198 entries, 0 to 1197
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1198 non-null   int64 
 1   text        1198 non-null   object
 2   keyword     1198 non-null   object
dtypes: int64(1), object(2)
memory usage: 28.2+ KB


In [ ]:
seededlda

,textID,ID,text,keyword,s1,s2,s3,s4,s5,s6,s7,s8,s9,label
0,1,1,자꾸 아 이건 다들 초등학교때 배웠을 텐데~ 하면서 설명 생략하시는데\n그거 고딩 ...,"자꾸,이건,다,들다,초등학교,때,배우다,하다,설명,생략,하다,그거,고딩,화,과정,늘...",0.034483,0.034483,0.172414,0.103448,0.034483,0.034483,0.034483,0.172414,0.379310,s9
1,2,2,시험이 11월에 끝나는 ㅂㅅ이 있다?,"시험,끝나다,이,있다",0.066667,0.066667,0.066667,0.200000,0.333333,0.066667,0.066667,0.066667,0.066667,s5
2,3,3,연애하고 싶다 이런건 아니고 (이미 남친 있음)\n\n최근에 축하 받을 일 생겼는데...,"연애,싶다,이렇다,아니다,이미,남친,있다,최근,축하,받다,일,생기다,남친,포항,있다...",0.054545,0.090909,0.090909,0.018182,0.018182,0.054545,0.127273,0.527273,0.018182,s8
3,4,4,이제 복전이라고 뜨게 주잖아... 그래도 회사에서 볼 때는 복전이라도 상경학위 있는...,"이제,복전,뜨다,주다,그래도,회사,볼,때,복전,상경,학위,있다,더,낫다,물론,스펙,...",0.021277,0.063830,0.148936,0.106383,0.489362,0.106383,0.021277,0.021277,0.021277,s5
4,5,5,오뚜기 정신을 본받아 숱한 실패에도 마음을 다잡고 다시 도전하는 사람이 멋진 사람이...,"오뚜기,정신,본받다,숱,실패,마음,다잡다,다시,도전,하다,사람,멋지다,사람,멋지다,...",0.010753,0.053763,0.032258,0.225806,0.010753,0.032258,0.591398,0.010753,0.032258,s7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,1194,1194,혹시 이런 기분 드는 사람 있어? 혼자 떨어져서 샤워할 때나 목욕할 때 드는데 갑자...,"혹시,이렇다,기분,들다,사람,있다,혼자,떨어지다,샤워,하다,때,목욕,하다,때,들다,...",0.052632,0.122807,0.228070,0.017544,0.052632,0.228070,0.122807,0.157895,0.017544,s3
1194,1195,1195,혹시 중도휴학도 신청할 수 있는 시기가 따로 있어?? 지금 2학기에 가게될 군대도 ...,"혹시,중도,휴학,신청,하다,수,있다,시기,따로,있다,지금,학기,가게,되다,군대,빼다...",0.042254,0.014085,0.014085,0.323944,0.098592,0.267606,0.014085,0.014085,0.211268,s4
1195,1196,1196,확인 강박증이 점점 심해져 현관문을 닫았는지 확인만 몇번해 내생각에 옛날에 집에 누...,"확인,강박증,점점,심해지다,현관문,닫다,확인,몇번,하다,생각,옛날,집,누가,들어오다...",0.032967,0.010989,0.076923,0.252747,0.054945,0.296703,0.010989,0.054945,0.208791,s6
1196,1197,1197,휴학하고 에타 처음 들어온다하는커뮤도 딱히 바쁜친구들에게 시덥잖은 얘기하기도그래서 ...,"휴학,에타,처음,들어오다,커뮤,딱하다,바쁘다,친구,시,덥다,얘기,하다,그래서,여기,...",0.087558,0.004608,0.032258,0.124424,0.078341,0.013825,0.096774,0.235023,0.327189,s9


In [ ]:
dep_word

,Unnamed: 0,word,tag,cleaned_word
0,0,가망이 없다,1,"가망,없다"
1,1,가슴 아프다,1,"가슴,아프다"
2,2,가치 없다,1,"가치,없다"
3,3,갇힌 것 같다,1,"갇히다,것,같다"
4,4,갑갑하다,1,갑갑하다
...,...,...,...,...
362,362,죽어야 한다,9,"죽다,하다"
363,363,죽음,9,죽음
364,364,칼빵,9,칼빵
365,365,한강에 뛰어들다,9,"한강,뛰어들다"


In [ ]:
"""everytime_keywords2 = everytime_keywords.dropna(axis=0)
everytime_keywords2 = everytime_keywords2.reset_index(drop=True)
print(len(everytime_keywords2))"""

'everytime_keywords2 = everytime_keywords.dropna(axis=0)\neverytime_keywords2 = everytime_keywords2.reset_index(drop=True)\nprint(len(everytime_keywords2))'

In [ ]:
everytime_keywords['keyword'] = [everytime_keywords['keyword'][i].split(',') for i in range(len(everytime_keywords['keyword']))]

In [ ]:
everytime_keywords

,Unnamed: 0,text,keyword
0,0,자꾸 아 이건 다들 초등학교때 배웠을 텐데~ 하면서 설명 생략하시는데\n그거 고딩 ...,"[자꾸, 이건, 다, 들다, 초등학교, 때, 배우다, 하다, 설명, 생략, 하다, ..."
1,1,시험이 11월에 끝나는 ㅂㅅ이 있다?,"[시험, 끝나다, 이, 있다]"
2,2,연애하고 싶다 이런건 아니고 (이미 남친 있음)\n\n최근에 축하 받을 일 생겼는데...,"[연애, 싶다, 이렇다, 아니다, 이미, 남친, 있다, 최근, 축하, 받다, 일, ..."
3,3,이제 복전이라고 뜨게 주잖아... 그래도 회사에서 볼 때는 복전이라도 상경학위 있는...,"[이제, 복전, 뜨다, 주다, 그래도, 회사, 볼, 때, 복전, 상경, 학위, 있다..."
4,4,오뚜기 정신을 본받아 숱한 실패에도 마음을 다잡고 다시 도전하는 사람이 멋진 사람이...,"[오뚜기, 정신, 본받다, 숱, 실패, 마음, 다잡다, 다시, 도전, 하다, 사람,..."
...,...,...,...
1193,1193,혹시 이런 기분 드는 사람 있어? 혼자 떨어져서 샤워할 때나 목욕할 때 드는데 갑자...,"[혹시, 이렇다, 기분, 들다, 사람, 있다, 혼자, 떨어지다, 샤워, 하다, 때,..."
1194,1194,혹시 중도휴학도 신청할 수 있는 시기가 따로 있어?? 지금 2학기에 가게될 군대도 ...,"[혹시, 중도, 휴학, 신청, 하다, 수, 있다, 시기, 따로, 있다, 지금, 학기..."
1195,1195,확인 강박증이 점점 심해져 현관문을 닫았는지 확인만 몇번해 내생각에 옛날에 집에 누...,"[확인, 강박증, 점점, 심해지다, 현관문, 닫다, 확인, 몇번, 하다, 생각, 옛..."
1196,1196,휴학하고 에타 처음 들어온다하는커뮤도 딱히 바쁜친구들에게 시덥잖은 얘기하기도그래서 ...,"[휴학, 에타, 처음, 들어오다, 커뮤, 딱하다, 바쁘다, 친구, 시, 덥다, 얘기..."


In [ ]:
dep_word2 = dep_word.dropna(axis=0)
dep_word2 = dep_word2.reset_index(drop=True)
print(len(dep_word2))

367


In [ ]:
dep_word2['cleaned_word'] = [dep_word2['cleaned_word'][i].split(',') for i in range(len(dep_word2['cleaned_word']))]

In [ ]:
dep_word2

,Unnamed: 0,word,tag,cleaned_word
0,0,가망이 없다,1,"[가망, 없다]"
1,1,가슴 아프다,1,"[가슴, 아프다]"
2,2,가치 없다,1,"[가치, 없다]"
3,3,갇힌 것 같다,1,"[갇히다, 것, 같다]"
4,4,갑갑하다,1,[갑갑하다]
...,...,...,...,...
362,362,죽어야 한다,9,"[죽다, 하다]"
363,363,죽음,9,[죽음]
364,364,칼빵,9,[칼빵]
365,365,한강에 뛰어들다,9,"[한강, 뛰어들다]"


In [ ]:
# seeded lda 결과에 전처리 결과 column 붙이기
seededlda['keywords'] = everytime_keywords['keyword']

In [ ]:
seededlda

,textID,ID,text,keyword,s1,s2,s3,s4,s5,s6,s7,s8,s9,label,keywords
0,1,1,자꾸 아 이건 다들 초등학교때 배웠을 텐데~ 하면서 설명 생략하시는데\n그거 고딩 ...,"자꾸,이건,다,들다,초등학교,때,배우다,하다,설명,생략,하다,그거,고딩,화,과정,늘...",0.034483,0.034483,0.172414,0.103448,0.034483,0.034483,0.034483,0.172414,0.379310,s9,"[자꾸, 이건, 다, 들다, 초등학교, 때, 배우다, 하다, 설명, 생략, 하다, ..."
1,2,2,시험이 11월에 끝나는 ㅂㅅ이 있다?,"시험,끝나다,이,있다",0.066667,0.066667,0.066667,0.200000,0.333333,0.066667,0.066667,0.066667,0.066667,s5,"[시험, 끝나다, 이, 있다]"
2,3,3,연애하고 싶다 이런건 아니고 (이미 남친 있음)\n\n최근에 축하 받을 일 생겼는데...,"연애,싶다,이렇다,아니다,이미,남친,있다,최근,축하,받다,일,생기다,남친,포항,있다...",0.054545,0.090909,0.090909,0.018182,0.018182,0.054545,0.127273,0.527273,0.018182,s8,"[연애, 싶다, 이렇다, 아니다, 이미, 남친, 있다, 최근, 축하, 받다, 일, ..."
3,4,4,이제 복전이라고 뜨게 주잖아... 그래도 회사에서 볼 때는 복전이라도 상경학위 있는...,"이제,복전,뜨다,주다,그래도,회사,볼,때,복전,상경,학위,있다,더,낫다,물론,스펙,...",0.021277,0.063830,0.148936,0.106383,0.489362,0.106383,0.021277,0.021277,0.021277,s5,"[이제, 복전, 뜨다, 주다, 그래도, 회사, 볼, 때, 복전, 상경, 학위, 있다..."
4,5,5,오뚜기 정신을 본받아 숱한 실패에도 마음을 다잡고 다시 도전하는 사람이 멋진 사람이...,"오뚜기,정신,본받다,숱,실패,마음,다잡다,다시,도전,하다,사람,멋지다,사람,멋지다,...",0.010753,0.053763,0.032258,0.225806,0.010753,0.032258,0.591398,0.010753,0.032258,s7,"[오뚜기, 정신, 본받다, 숱, 실패, 마음, 다잡다, 다시, 도전, 하다, 사람,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,1194,1194,혹시 이런 기분 드는 사람 있어? 혼자 떨어져서 샤워할 때나 목욕할 때 드는데 갑자...,"혹시,이렇다,기분,들다,사람,있다,혼자,떨어지다,샤워,하다,때,목욕,하다,때,들다,...",0.052632,0.122807,0.228070,0.017544,0.052632,0.228070,0.122807,0.157895,0.017544,s3,"[혹시, 이렇다, 기분, 들다, 사람, 있다, 혼자, 떨어지다, 샤워, 하다, 때,..."
1194,1195,1195,혹시 중도휴학도 신청할 수 있는 시기가 따로 있어?? 지금 2학기에 가게될 군대도 ...,"혹시,중도,휴학,신청,하다,수,있다,시기,따로,있다,지금,학기,가게,되다,군대,빼다...",0.042254,0.014085,0.014085,0.323944,0.098592,0.267606,0.014085,0.014085,0.211268,s4,"[혹시, 중도, 휴학, 신청, 하다, 수, 있다, 시기, 따로, 있다, 지금, 학기..."
1195,1196,1196,확인 강박증이 점점 심해져 현관문을 닫았는지 확인만 몇번해 내생각에 옛날에 집에 누...,"확인,강박증,점점,심해지다,현관문,닫다,확인,몇번,하다,생각,옛날,집,누가,들어오다...",0.032967,0.010989,0.076923,0.252747,0.054945,0.296703,0.010989,0.054945,0.208791,s6,"[확인, 강박증, 점점, 심해지다, 현관문, 닫다, 확인, 몇번, 하다, 생각, 옛..."
1196,1197,1197,휴학하고 에타 처음 들어온다하는커뮤도 딱히 바쁜친구들에게 시덥잖은 얘기하기도그래서 ...,"휴학,에타,처음,들어오다,커뮤,딱하다,바쁘다,친구,시,덥다,얘기,하다,그래서,여기,...",0.087558,0.004608,0.032258,0.124424,0.078341,0.013825,0.096774,0.235023,0.327189,s9,"[휴학, 에타, 처음, 들어오다, 커뮤, 딱하다, 바쁘다, 친구, 시, 덥다, 얘기..."


In [ ]:
"""seed_dic = {1 : ["혼자", "기분", "싫다","아프다", "마음", "행복하다", "우울증", "우울하다"],
            2 : ["공부", "싫다"],
            3 : ["버리다", "먹다", "약", "안", "많이", "살"],
            4 : ["자다", "많이", "잠", "안"],
            5 : ["시간", "공부", "시험", "무섭다"],
            6 : ["공부", "만나다"], 
            7 : ["혼자", "친구", "동생", "부모님"],
            8 : ["공부"],
            9 : ["자살", "살다", "살", "죽다"]}"""

In [ ]:
"""seed_dic= {1: ["아프다", "기분", "더", "마음", "싫다", "우울하다", "죽다", "수"],
           2: ["안", "싫다", "나다", "일"],
           3: ["또", "먹다", "버리다", "모르다", "싫다", "안", "맞다"],
           4: ["자다", "잠", "오다", "많다"],
           5: ["말", "거", "자다", "나오다", "무섭다"],
           6: ["기분", "안", "싫다"],
           7: ["나다", "안", "왜", "싫다", "받다", "나르다", "살다"],
           8: ["나다", "안"],
           9:["자살", "살다", "살", "싫다", "뭐", "안", "아프다", "죽다", "먹다", "이제", "잠", "괜찮다"]}"""

In [ ]:
seed_dic = {1: ["아프다",   "싫다",     "우울하다", "행복하다"],
            2: ["감정", "싫다", "나다", "집"],
            3: ["또",     "먹다",   "버리다", "싫다",   "맞다"],
            4: ["못",   "자다", "다시", "자꾸"],
            5: ["자다",   "무섭다"],
            6: ["집",   "싫다"],
            7: ["나다",   "왜",     "싫다",   "나르다", "살다" ],
            8: ["나다"],
            9: ["못",     "살다",  "살",   "싫다",  "뭐",   "하나", "아프다", "약",     "먹다"  ]}

In [ ]:
seededlda.rename(columns ={'s1':1, 's2':2, 's3':3, 's4':4,	's5':5	,'s6':6,	's7':7,	's8':8,	's9':9}, inplace=True)

In [ ]:
seededlda

,textID,ID,text,keyword,1,2,3,4,5,6,7,8,9,label,keywords
0,1,1,자꾸 아 이건 다들 초등학교때 배웠을 텐데~ 하면서 설명 생략하시는데\n그거 고딩 ...,"자꾸,이건,다,들다,초등학교,때,배우다,하다,설명,생략,하다,그거,고딩,화,과정,늘...",0.034483,0.034483,0.172414,0.103448,0.034483,0.034483,0.034483,0.172414,0.379310,s9,"[자꾸, 이건, 다, 들다, 초등학교, 때, 배우다, 하다, 설명, 생략, 하다, ..."
1,2,2,시험이 11월에 끝나는 ㅂㅅ이 있다?,"시험,끝나다,이,있다",0.066667,0.066667,0.066667,0.200000,0.333333,0.066667,0.066667,0.066667,0.066667,s5,"[시험, 끝나다, 이, 있다]"
2,3,3,연애하고 싶다 이런건 아니고 (이미 남친 있음)\n\n최근에 축하 받을 일 생겼는데...,"연애,싶다,이렇다,아니다,이미,남친,있다,최근,축하,받다,일,생기다,남친,포항,있다...",0.054545,0.090909,0.090909,0.018182,0.018182,0.054545,0.127273,0.527273,0.018182,s8,"[연애, 싶다, 이렇다, 아니다, 이미, 남친, 있다, 최근, 축하, 받다, 일, ..."
3,4,4,이제 복전이라고 뜨게 주잖아... 그래도 회사에서 볼 때는 복전이라도 상경학위 있는...,"이제,복전,뜨다,주다,그래도,회사,볼,때,복전,상경,학위,있다,더,낫다,물론,스펙,...",0.021277,0.063830,0.148936,0.106383,0.489362,0.106383,0.021277,0.021277,0.021277,s5,"[이제, 복전, 뜨다, 주다, 그래도, 회사, 볼, 때, 복전, 상경, 학위, 있다..."
4,5,5,오뚜기 정신을 본받아 숱한 실패에도 마음을 다잡고 다시 도전하는 사람이 멋진 사람이...,"오뚜기,정신,본받다,숱,실패,마음,다잡다,다시,도전,하다,사람,멋지다,사람,멋지다,...",0.010753,0.053763,0.032258,0.225806,0.010753,0.032258,0.591398,0.010753,0.032258,s7,"[오뚜기, 정신, 본받다, 숱, 실패, 마음, 다잡다, 다시, 도전, 하다, 사람,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,1194,1194,혹시 이런 기분 드는 사람 있어? 혼자 떨어져서 샤워할 때나 목욕할 때 드는데 갑자...,"혹시,이렇다,기분,들다,사람,있다,혼자,떨어지다,샤워,하다,때,목욕,하다,때,들다,...",0.052632,0.122807,0.228070,0.017544,0.052632,0.228070,0.122807,0.157895,0.017544,s3,"[혹시, 이렇다, 기분, 들다, 사람, 있다, 혼자, 떨어지다, 샤워, 하다, 때,..."
1194,1195,1195,혹시 중도휴학도 신청할 수 있는 시기가 따로 있어?? 지금 2학기에 가게될 군대도 ...,"혹시,중도,휴학,신청,하다,수,있다,시기,따로,있다,지금,학기,가게,되다,군대,빼다...",0.042254,0.014085,0.014085,0.323944,0.098592,0.267606,0.014085,0.014085,0.211268,s4,"[혹시, 중도, 휴학, 신청, 하다, 수, 있다, 시기, 따로, 있다, 지금, 학기..."
1195,1196,1196,확인 강박증이 점점 심해져 현관문을 닫았는지 확인만 몇번해 내생각에 옛날에 집에 누...,"확인,강박증,점점,심해지다,현관문,닫다,확인,몇번,하다,생각,옛날,집,누가,들어오다...",0.032967,0.010989,0.076923,0.252747,0.054945,0.296703,0.010989,0.054945,0.208791,s6,"[확인, 강박증, 점점, 심해지다, 현관문, 닫다, 확인, 몇번, 하다, 생각, 옛..."
1196,1197,1197,휴학하고 에타 처음 들어온다하는커뮤도 딱히 바쁜친구들에게 시덥잖은 얘기하기도그래서 ...,"휴학,에타,처음,들어오다,커뮤,딱하다,바쁘다,친구,시,덥다,얘기,하다,그래서,여기,...",0.087558,0.004608,0.032258,0.124424,0.078341,0.013825,0.096774,0.235023,0.327189,s9,"[휴학, 에타, 처음, 들어오다, 커뮤, 딱하다, 바쁘다, 친구, 시, 덥다, 얘기..."


In [ ]:
seededlda.loc[0][2]

0.03448275862068965

In [ ]:
# 라벨 부여 기준 1
labels = []
for i in range(len(seededlda)):
  data = seededlda.loc[i]
  data_label = []
  
  for label in range(1,10):
    if list(set(data['keywords']).intersection(seed_dic[label])) or data[label]>0.4:
      data_label.append(label)
  labels.append(data_label)


In [ ]:
# 라벨 부여 기준 2
labels = []
for i in range(len(seededlda)):
  data = seededlda.loc[i]
  data_label = []
  
  for label in range(1,10):
    if list(set(data['keywords']).intersection(seed_dic[label])) and data[label]>0.13:
      data_label.append(label)
  labels.append(data_label)


In [ ]:
result = {'text': seededlda['text'], 'label': labels}
result_df = pd.DataFrame(result)
result_df

,text,label
0,자꾸 아 이건 다들 초등학교때 배웠을 텐데~ 하면서 설명 생략하시는데\n그거 고딩 ...,[]
1,시험이 11월에 끝나는 ㅂㅅ이 있다?,[]
2,연애하고 싶다 이런건 아니고 (이미 남친 있음)\n\n최근에 축하 받을 일 생겼는데...,[]
3,이제 복전이라고 뜨게 주잖아... 그래도 회사에서 볼 때는 복전이라도 상경학위 있는...,[]
4,오뚜기 정신을 본받아 숱한 실패에도 마음을 다잡고 다시 도전하는 사람이 멋진 사람이...,"[4, 7]"
...,...,...
1193,혹시 이런 기분 드는 사람 있어? 혼자 떨어져서 샤워할 때나 목욕할 때 드는데 갑자...,[]
1194,혹시 중도휴학도 신청할 수 있는 시기가 따로 있어?? 지금 2학기에 가게될 군대도 ...,[9]
1195,확인 강박증이 점점 심해져 현관문을 닫았는지 확인만 몇번해 내생각에 옛날에 집에 누...,"[4, 6, 9]"
1196,휴학하고 에타 처음 들어온다하는커뮤도 딱히 바쁜친구들에게 시덥잖은 얘기하기도그래서 ...,[9]


In [ ]:
for i in range(len(result_df)):
  data = result_df.loc[i]
  if 1 in data['label']:
    print("-"*20)
    print(data['text'])
    print(data['label'])

--------------------
아니 좀 내가 싫다고 의사표현을 하면 안 했으면 좋겠어. 아무리 부모라도. 아무리 날 낳은 주인이더라도.

내가 싫다고 한 두 번 얘기했으면 좀 들었으면 좋겠어. 100번 넘게 얘기해도 비웃거나 장난치며 반응하는 거 죽이고 싶어. 사람 미치게 만드는 건 진짜 한 재주 하는 것 같아.

벽에다 얘기하는 것보다 훨씬 사람을 미치게 해.
[1, 2]
--------------------
내가 이 게시판을 반년만에 들어오다니 아직 우울하긴 해도 많이 건강해졌구나 나
[1]
--------------------
요즘 왤케 우울하냐

돈 써도 잠깐
맛있는 걸 먹어도 잠깐
어딜 놀러가도 잠깐 뿐이네
[1, 3]
--------------------
힘들다는 정의를 모르겠어요
어느 선을 넘어야 힘들다고 말할 수 있는지
세상에 힘들지 않은 사람은 없는데
힘들다고 말하면
어린 애가 되는 것만 같아요
습관적으로 숨기고 억누르고 하니까
괜찮아지긴 하는 것 같더라고요
작은 일에서도 행복함을 느끼고 감사함을 느끼기도 하고요
근데 여전히 변함은 없어요
역시 질질 이어가는 것 보다 끝내는 게 맞는 것 같아요
그게 언제쯤일까 기다리면서
하루하루 살아가는 것 같아요
[1, 3]
--------------------
어릴 때부터 공부공부, 대학교 잘 가야한다 공부공부
그래서 재수를 하게 됐고 극심한 우울증에 자해 불면 자살시도 별 나쁜 건 다 겪었는데
대학교에 왔더니 다시 공부 공부
이제 너는 공부만 하면 된다 뒷바라지는 자기가 하겠다

이 말이 얼마나 나를 불효새끼로 만드는 지 모르겠다
공부가 그렇게 중요하면 자기들이나 하지
그래놓고 나중에 힘들고 우울하면 뭐가 그렇게 만든 건지
지들끼리 또 탐색하지
그때가 되면 나는 어떻게 당신들 때문에 우울하고 괴롭냐고 말합니까
[1, 9]
--------------------
기분 안좋을때 초콜릿 먹으면 달달함에 마비돼서 잠깐동안 덜 괴로웠는데
이제 초콜릿 먹어도 힘들다
내가 혐오스러워서 죽고싶어..... 어제는 답

In [ ]:
for i in range(len(result_df)):
  data = result_df.loc[i]
  if 5 in data['label']:
    print("-"*20)
    print(data['text'])
    print(data['label'])

--------------------
오뚜기 정신을 본받아 숱한 실패에도 마음을 다잡고 다시 도전하는 사람이 멋진 사람이다.
멋진 사람이 되고자 멋져 보이는 사람들 곁에 있으려 노력했고 스스로도 부단히 반성했지만 능력 부족으로 스스로에 대한 확신도, 비전도, 끈기도, 열정도 사라져 스스로가 생각하기에 가장 혐오스러운 회색 사람이 되어 버렸다.
자타공인의 좋은 피드백을 받을 정도로 잘 해야 하는데 그렇지 못함에 슬프다.
과정 속에 있어서 온전한 결과물이 어떨지 알지 못하기에 더더욱 타인 앞에 나설 수 없고 위축되는 것 같다.
자꾸만 늦잠을 자고 싶고
이불 속에서 나오기 싫고
밖으로 도망가거나
책상 밑으로 숨고 싶은 건
이런 이유에서일 거라 생각한다.
[1, 5, 7]
--------------------
내가 워낙 완벽주의자이기도하고 학업, 예체능 등을 포함한 여러 면에서 오빠보다 출중하단말이야 (객관적으로 다들 그렇게 말해, 가족제외)
그래서 그런지 엄마는 더 오빠 기 세워주려고 하는 말인지 맨날 오빠는 손재주가 참 좋다, 성격이 참 좋다, 재능이 있다,, 이렇게 직접적으로 좋은 말을 많이 해주는데
나한테는 절대 칭찬 안 해줘, 공부 잘해도 그냥 그러려니, 다른 것들도 남이 엄마한테 " **이 참 이것저것 다 잘한다~" 라고 말하면 항상 "**이는 뭐든 열심히 해, 노력하는 스타일이야" 이런식으로만 말해.
물론 맞는 말이야, 난 어릴적부터 애정결핍이였던 것 같고 그래서 더 열심히 살았고 뭐든 오빠보다는 잘 하려고 했었어, 노력파가 맞아, 하지만 어른이 되었고 시간이 많이 흘렀는데도 엄마한테 항상 서운한 마음이야. 그래서 더 틱틱대고 소통하기 싫어져, 가장 나를 아껴주고 보듬어줘야 할 사람이 되려 내 자존감 깎고 있는 것 같아서 솔직히 마주보고싶지도 않아.
그런데 엄마도 마음이 여려서 요즘 우울해보인단 말이지, 그 모습을 내가 또 그냥 못봐서 잘해주고 싶어해, 그런데 내가 좋은 곳 데려가고 기쁘게 해주려 해도 엄마는 뭐든 오빠랑 하고싶어하는 것 같고 나랑 있는 

In [ ]:
count = 0
for i in range(len(result_df)):
  data = result_df.loc[i]
  if 0 in data['label']:
    count+=1
    print("-"*20)
    print(data['text'])
    print(data['label'])

In [ ]:
result_df.to_csv('/content/drive/MyDrive/2023_연구학점/jiae/multilabeled_everytime0603_3.csv', sep=',')

In [ ]:
count

0